In [ ]:
!pip install wandb

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
wandb.init(project="Fashion_Mnist", entity="gharabadiyan")

wandb: Currently logged in as: gharabadiyan. Use `wandb login --relogin` to force relogin


In [5]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()

X_test = X_test / 255.0

X_train,X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=24)

X_train = X_train / 255.0



4431872/4422102 [==============================] - 0s 0us/step


## MLP

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=12,validation_data=(X_val,Y_val),callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/12
1500/1500 [==============================] - 7s 3ms/step - loss: 0.5162 - accuracy: 0.8190 - val_loss: 62.8551 - val_accuracy: 0.8370 - _timestamp: 1654668301.0000 - _runtime: 46.0000
Epoch 2/12
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3907 - accuracy: 0.8584 - val_loss: 54.8722 - val_accuracy: 0.8578 - _timestamp: 1654668305.0000 - _runtime: 50.0000
Epoch 3/12
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3485 - accuracy: 0.8735 - val_loss: 49.4740 - val_accuracy: 0.8605 - _timestamp: 1654668309.0000 - _runtime: 54.0000
Epoch 4/12
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3214 - accuracy: 0.8812 - val_loss: 58.2844 - val_accuracy: 0.8522 - _timestamp: 1654668312.0000 - _runtime: 57.0000
Epoch 5/12
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3038 - accuracy: 0.8887 - val_loss: 48.6955 - val_accuracy: 0.8624 - _timestamp: 1654668316.0000 - _runtime: 61.0000
Epoch 6/12
1500/1500 [===

In [7]:
model.evaluate(X_test,  Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.3525 - accuracy: 0.8798


[0.3524978458881378, 0.879800021648407]

In [8]:
model.save('MLP_fashionMnist.h5')

## MLP + CNN

In [10]:
model = tf.keras.models.Sequential([
                                    
        layers.Conv2D(32,(3,3),activation='relu',input_shape=(28, 28,1)),
        layers.MaxPool2D(),
        layers.Conv2D(64,(3,3),activation='relu'),                            
        layers.MaxPool2D(),
        layers.Conv2D(32,(5,5),activation='relu'),

        layers.Flatten(),

        
        tf.keras.layers.Dense(48, activation='relu'),
        tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
model.fit(X_train,Y_train,epochs=10,validation_data=(X_val,Y_val))

Epoch 1/10
1500/1500 [==============================] - 15s 4ms/step - loss: 0.5270 - accuracy: 0.8079 - val_loss: 58.4470 - val_accuracy: 0.8297
Epoch 2/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3512 - accuracy: 0.8721 - val_loss: 48.5500 - val_accuracy: 0.8417
Epoch 3/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3023 - accuracy: 0.8891 - val_loss: 43.2785 - val_accuracy: 0.8512
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2664 - accuracy: 0.9024 - val_loss: 41.3848 - val_accuracy: 0.8498
Epoch 5/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2425 - accuracy: 0.9113 - val_loss: 54.6790 - val_accuracy: 0.8282
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2192 - accuracy: 0.9194 - val_loss: 64.9007 - val_accuracy: 0.7940
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2025 - accuracy: 0.9247 - val_loss: 40.1708 - val_a

In [12]:
model.evaluate(X_test,Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.2787 - accuracy: 0.9049


[0.27873384952545166, 0.9049000144004822]

In [13]:
model.save('CNN_MLP_fashionMnist.h5')